## **_Workspace_**

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup as bs
import yfinance as yf
import numpy as np

### **_DataLoader_**

In [4]:
# DataLoader class
class DataLoader():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Live data scraper
    def Live(self, symbol, choice):
        
		# Checking if price is chosen
        if choice == "price":
            
			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"
            
			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")
            
			# Finding and returning the price
            return float(htmlcode.find("fin-streamer", {"class": "livePrice svelte-mgkamr"}).text.replace(",", ''))

		# Checking if market capitalisation is chosen
        elif choice == "mcap":

			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"

			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")

            # Extracting the market capitalisation expression
            cap = (htmlcode.find("fin-streamer", {"data-field": "marketCap"}).text.replace(" ", ''))    
            
			# Creating a dictionary of suffix multipliers
            multipliers = {
            	'K': 1000,
            	'M': 1000000,
        		'B': 1000000000,
        		'T': 1000000000000
            }
            
			# Checking if there is suffix on the extracted expression
            if cap[-1] in multipliers:
                
				# Extracting and using the multiplier
                multiplier = multipliers[cap[-1]]
                value = float(cap[:-1]) * multiplier
                return int(value)
            
			# Otherwise there's no suffix so returning the raw extraction
            else:
                
                # Converting the cap to a float
                return float(cap)
            
    # Past data scraper
    def Past(self, universe):

		# Download and extract the close data
        data = yf.download(universe, start=datetime.today()-timedelta(30), end=datetime.today())
        closes = data["Close"]
        
		# Returning the close prices
        return closes

### **_Stock Screener_**

In [5]:
# Stock screener class
class Screener():
    
	# Initialiser
    def __init__(self):
        pass
        
	# Method to actually screen the stocks
    def Screen(self, data, count):
        
		# Getting the bounds
        before=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[0]
        after=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[-1]
        
		# Calculating the monthly returns
        returns = (((after-before) / before) * 100).sort_values(ascending=False)
        
		# Filtering out the best performing
        return returns.iloc[:count]

### **_Allocator_**

In [6]:
# Allocator class
class Allocator():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method to allocate capital to stcoks according to Mcap weighting
    def Allocate(self, screened, capital, loader):
        
        # Creating lists for storage of prices and caps
        prices = []
        caps = []

		# Looping over the keys to extract prices for each
        for index in range(len(screened)):
            
			# Updating the lists
            prices.append(loader.Live(screened.keys()[index], "price"))
            caps.append(loader.Live(screened.keys()[index], "mcap"))

        # Converting the lists to arrays
        prices = np.array(prices)
        caps = np.array(caps)

		# Calculations for the respective quantities
        totalcap = sum(caps)
        proportions = caps / totalcap
        allocations = proportions * capital
        quantities = allocations // prices

		# Sorting everything into a dataframe
        details = pd.DataFrame({"Symbols":screened.keys(),
                                "Market Cap":caps,
                                "Proportions":proportions,
                                "Allocations":allocations,
                                "Quantities":quantities,
                                "Prices":prices})
        
		# Returning this dataframe to the user
        return details

### **_OrderManager_**

In [7]:
# Order manager class
class OrderManger():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method to create an order
    def MakeOrder(self, symbol, quantity, side):
        
		# Creating a data loader and extracting the live price
        dataloader = DataLoader()
        price = dataloader.Live(symbol, "price")
        
		# Getting today's date and time
        date = datetime.today()
        
		# Creating an order bracket
        bracket = pd.DataFrame({"Date":[date],
                                "Symbol":[symbol],
                                "Price":[price],
                                "Quantity":[quantity],
                                "Action":[side]})
        
		# Returning an order dataframe for the user
        return bracket

### **_Rebalance_**

In [327]:
# Rebalancing the portfolio
class Rebalance():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method for running the TLH engine
    def Engine(self, trades, assets):
        
        # Calculate short and long term gains for each of those symbol
        def CalculateGains(trades, assets):
            
			# These lists and variables store gains data
            total_long_term_gains = 0
            total_short_term_gains = 0
            individual_long_term_gains = []
            individual_short_term_gains = []
    
            # Looping over the assets
            for i in range(len(assets)):

                # Passing the filler value
                if assets.iloc[i]["Symbol"] != "filler":

				    # Extracting the symbol and its respective trades
                    current_symbol = assets.iloc[i]["Symbol"]
                    current_trades = trades[trades["Symbol"] == current_symbol]
                    
					# Initialise different gains storage
                    short_term_gains = 0
                    long_term_gains = 0
    
                    # These lists contain their respective long positions
                    all_long_positions = []
                    short_term_long_positions = []
                    long_term_long_positions = []

	    			# Looping over all trades
                    for i in range(len(current_trades)):
	    		
	    				# Extracting a row for easier indexing
                        row = current_trades.iloc[i]
	    		
	    				# Checking if it's a buy
                        if row["Action"] == "buy":
	    			
	    					# Appending all buys no matter the term
                            all_long_positions.append(row.copy())
	    			
	    				# Checking if it's a sell
                        elif row["Action"] == "sell":
	    			
	    					# Gathering specific sell details
                            og_shares_sold = row["Quantity"]
                            shares_sold = row["Quantity"]
                            price_sold = row["Price"]
                            date_sold = row["Date"]
                            total_short_term_cost_basis = 0
                            total_long_term_cost_basis = 0

	    					# Checking for long term or short term buys
                            for j in range(len(all_long_positions)):
	    				
	                            # If the purchase was made more than a year before the sale
                                if all_long_positions[j]["Date"] < (date_sold - timedelta(365)):
	    					
	    	                        # Adding to the long term list
                                    long_term_long_positions.append(j)

	    						# Otherwise                
                                else:
	    					
	    							# Adding to the short term list
                                    short_term_long_positions.append(j)

	    					# Process the shares sold for long term
                            while shares_sold > 0 and long_term_long_positions:
	    			
	    						# Extracting the oldest long term buy
                                oldest_long_term_buy_index = long_term_long_positions[0]
                                oldest_long_term_buy = all_long_positions[oldest_long_term_buy_index]
	    				
	    						# Checking if there are more shares sold than the quantity of the oldest buy
                                if shares_sold >= oldest_long_term_buy["Quantity"]:
	    				
	    							# Calculating the cost basis
                                    total_long_term_cost_basis += (oldest_long_term_buy["Quantity"] * oldest_long_term_buy["Price"])
	    					
	    							# Updating the amount of shares left to handle
                                    shares_sold -= oldest_long_term_buy["Quantity"]
	    					
	    							# All oldest shares were used so they can be discarded
                                    long_term_long_positions.pop(0)
	    				
	    						# Otherwise deal with remaining shares
                                else:
	    					
	    							# Calculating the basis as the remaining shares to sell times oldest price
                                    total_long_term_cost_basis += (shares_sold * oldest_long_term_buy["Price"])
	    					
	    							# Updating the amount of shares left on the oldest buy
                                    all_long_positions[oldest_long_term_buy_index]["Quantity"] -= shares_sold

	    							# No shares left to sell so setting it to zero which exits the loop
                                    shares_sold = 0

	    					# Total long term proceeds - the accurate cost basis
                            long_term_sold = og_shares_sold - shares_sold
                            long_term_proceeds = long_term_sold * price_sold
                            long_term_gains += long_term_proceeds - total_long_term_cost_basis
	    			
                            # Reset shares sold for short-term calculation
                            shares_sold = og_shares_sold - long_term_sold

	    					# Process the shares sold for short term
                            while shares_sold > 0 and short_term_long_positions:
	    				
	    						# Extracting the oldest short term buy
                                oldest_short_term_buy_index = short_term_long_positions[0]
                                oldest_short_term_buy = all_long_positions[oldest_short_term_buy_index]
	    				
	    						# Checking if there are more shares sold than the quantity of the oldest buy
                                if shares_sold >= oldest_short_term_buy["Quantity"]:
	    					
	    							# Calculating the cost basis
                                    total_short_term_cost_basis += (oldest_short_term_buy["Quantity"] * oldest_short_term_buy["Price"])
	    					
	    							# Updating the amount of shares left to handle
                                    shares_sold -= oldest_short_term_buy["Quantity"]
	    					
	    							# All oldest shares were used so they can be discarded
                                    short_term_long_positions.pop(0)
	    				
	    						# Otherwise deal with remaining shares
                                else:
	    					
	    							# Calculating the basis as the remaining shares to sell times oldest price
                                    total_short_term_cost_basis += (shares_sold * oldest_short_term_buy["Price"])
	    					
	    							# Updating the amount of shares left on the oldest buy
                                    all_long_positions[oldest_short_term_buy_index]["Quantity"] -= shares_sold

	    							# No shares left to sell so setting it to zero which exits the loop
                                    shares_sold = 0

	    					# Total short term proceeds - the accurate cost basis
                            short_term_sold = og_shares_sold - long_term_sold
                            short_term_proceeds = short_term_sold * price_sold
                            short_term_gains += short_term_proceeds - total_short_term_cost_basis
                    
					# Excluding the losses as if they're unrealised
                    if long_term_gains > 0:
                        total_long_term_gains += long_term_gains
                    if short_term_gains > 0:
                        total_short_term_gains += short_term_gains
                    individual_long_term_gains.append(long_term_gains)
                    individual_short_term_gains.append(short_term_gains)
                    
				# Otherwise adding the filler value
                else:
                    
					# Appending the filler
                    individual_long_term_gains.append("filler")
                    individual_short_term_gains.append("filler")
                    
			# After the loop update the assets to include respective gains or losses
            assets["LT Gains"] = individual_long_term_gains
            assets["ST Gains"] = individual_short_term_gains
                    
            # Might change to flooring the decimal
            return round(total_long_term_gains, 2), round(total_short_term_gains, 2), assets

        # Filtering out the positions at losses
        def FindLosses(assets):
            
            # Boolean mask storage
            mask = []

            # Filter them out
            for i in range(len(assets)):
                
				# Passing the filler value
                if assets.iloc[i]["Symbol"] != "filler":
                    
					# Checking if the long term gains
                    if assets.iloc[i]["LT Gains"] < 0 or assets.iloc[i]["ST Gains"] < 0:
                        
						# Updating the mask
                        mask.append(True)
                        
					# Otherwise
                    else:
                        
						# Updating the mask
                        mask.append(False)
                        
				# Otherwise
                else:
                    
					# Updating the mask
                    mask.append(False)

			# Returning the mask to filter       
            return mask
                    
        # Filtering out violations of wash sale rule
        def WashSale(assets, trades):
            
			# Boolean mask storage
            mask = []
            
			# Looping over the already filtered assets
            for i in range(len(assets)):
                    
				# Extracting specifics
                current_symbol = assets.iloc[i]["Symbol"]
                current_trades = trades[trades["Symbol"] == current_symbol]
                
                # Doesn't violate yet
                is_safe = True
                    
				# Looping over the trades of a specific symbol
                for j in range(len(current_trades)):
                        
					# Checking if the trade falls in the wash sale period
                    if (datetime.today() - timedelta(30)) <= current_trades["Date"].iloc[j] <= (datetime.today() + timedelta(30)):
                            
						# Violation
                        is_safe = False
                        break
                    
				# Updating the mask with respective boolean
                mask.append(is_safe)
                    
			# Returning the mask
            return mask

        # Calculating tax due before and after
        def CalculateTax(assets, long_term_gains, short_term_gains):
            
			# Long term tax before
            if long_term_gains > 0:
                long_term_tax_before = 0.1 * long_term_gains
            else:
                long_term_tax_before = 0
                
			# Short term tax before
            if short_term_gains > 0:
                short_term_tax_before = 0.2 * short_term_gains
            else:
                short_term_tax_before = 0
                
			# Calculate the total tax before
            total_tax_before = long_term_tax_before + short_term_tax_before

			# Calculate total losses to offset the gains
            total_long_term_losses = assets["LT Gains"].sum()
            total_short_term_losses = assets["ST Gains"].sum()
            
            # Net gains after offsetting losses
            net_long_term_gains = total_long_term_gains + total_long_term_losses
            net_short_term_gains = total_short_term_gains + total_short_term_losses
            
			# Ensure that net gains cannot be negative
            net_long_term_gains = max(net_long_term_gains, 0)
            net_short_term_gains = max(net_short_term_gains, 0)

			# Long term tax after
            if net_long_term_gains > 0:
                long_term_tax_after = 0.1 * net_long_term_gains
            else:
                long_term_tax_after = 0
                
            # Short term tax after
            if net_short_term_gains > 0:
                short_term_tax_after = 0.2 * net_short_term_gains
            else:
                short_term_tax_after = 0
                
			# Calculating the total tax before and after the harvest
            total_tax_after = long_term_tax_after + short_term_tax_after
            print(f"Total tax before: ${total_tax_before}")
            print(f"Total tax after:  ${total_tax_after}")
            
			# Calculating the savings
            if total_tax_before - total_tax_after > 0:
                print(f"Savings after harvest: ${total_tax_before - total_tax_after}")
                return assets["Symbol"]

        # Calculating the gains and losses of each symbol and the totals
        total_long_term_gains, total_short_term_gains, assets = CalculateGains(trades, assets)
        print(f"LT Gains: ${total_long_term_gains}")
        print(f"ST Gains: ${total_short_term_gains}")
        print()
        print("All Assets:")
        print(assets)
        print()
        loss_mask = FindLosses(assets)
        assets_at_loss = assets[loss_mask]           
        print("Assets At Loss:")
        print(assets_at_loss)
        print()
        safe_mask = WashSale(assets_at_loss, trades)
        print("Safe Assets")
        print(assets_at_loss[safe_mask])
        print()
        if total_long_term_gains+total_short_term_gains > 0:
            if assets_at_loss[safe_mask].empty == False:
                harvest = CalculateTax(assets_at_loss[safe_mask], total_long_term_gains, total_short_term_gains)
                print()
                print(harvest)
                return harvest
            else:
                print("No losses to harvest")
                return None
        else:
            print("No gains to be taxed")
            return None

### **_Portfolio_**

In [340]:
# Portfolio class
class Portfolio():
    
	# Initialiser
    def __init__(self, universe, capital, size):
        
		# Setting portfolio attributes
        self.size = size
        self.capital = capital
        self.universe = universe.copy()
        self.available_universe = universe.copy()
        
		# Setting manager dataframes for tracking trades and assets
        self.trades = pd.DataFrame({"Date":["filler", datetime(2021,1,1), datetime(2022,3,4), datetime(2023,1,1), datetime(2023,2,2)],
                                    "Symbol":["filler", "SOFI", "SOFI", "AAPL", "AAPL"],
                                    "Price":["filler", 200, 250, 200, 240],
                                    "Quantity":["filler", 10, 10, 10, 10],
                                    "Action":["filler", "buy", "sell", "buy", "sell"]})
        self.assets = pd.DataFrame({"Symbol":["filler"], "Quantity":["filler"], "LT Gains":["filler"], "ST Gains":["filler"]})
        self.manager = OrderManger()

		# Objects for choosing the top stocks of our universe
        loader = DataLoader()
        screener = Screener()
        allocator = Allocator()
        
		# Calculating quantities for what stocks
        stocks = loader.Past(self.universe)
        screened = screener.Screen(stocks, self.size)
        allocated = allocator.Allocate(screened, self.capital, loader)
        
		# Looping over the chosen stocks and adding them to the portfolio
        for index in range(len(allocated)):
            
			# Adding the asset
            self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
        
	# Method to buy an asset for the portfolio
    def BuyAsset(self, symbol, quantity):
        
		# Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "buy")
        
		# If the asset isn't already present on the portfolio
        if order["Symbol"][0] not in self.assets["Symbol"].values:
            
			# Checking if a new stock breaches the portfolio size
            if (len(self.assets)-1) >= self.size:
                print("Portfolio size limit reached. Cannot add new asset")
                return
            
			# Adding the asset to the portfolio asset list
            update = pd.DataFrame({"Symbol":[order["Symbol"][0]], "Quantity":[order["Quantity"].values[0]]})
            self.assets = pd.concat([self.assets, update]).reset_index(drop=True)
            
			# Logging the trade
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
            
			# Updating the list of available stocks that aren't under the portfolio
            self.available_universe.remove(order["Symbol"][0])
        
		# Otherwise if the asset is already in the portfolio
        else:
            
			# Updating the quantity of the assets
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity + order["Quantity"].values[0]
            self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
            
			# Logging the other trades
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)

		# Updating the capital
        self.capital -= (order["Quantity"] * order["Price"])
    
	# Method to sell an asset for the portfolio
    def SellAsset(self, symbol, quantity):
        
        # Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "sell")

		# If the asset is already present on the portfolio
        if order["Symbol"][0] in self.assets["Symbol"].values:
            
			# Calculating the new portfolio quantity of the asset
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity - order["Quantity"].values[0]
            
			# Checking if the sale quantity was greater than current quantity (Short sale)
            if new_quantity < 0:
                
				# Inform the user and exit
                print("Quantity too great, no short sales allowed")
            
			# If the sale cancels out position completely then remove it from assets
            elif new_quantity == 0:
                
				# Removing the asset from the portfolio
                self.assets = self.assets.loc[self.assets["Symbol"] != order["Symbol"][0]].reset_index(drop=True)
                
				# Logging the trade
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += (order["Quantity"] * order["Price"])
                
				# Updating the stocks that we don't have under the portfolio
                self.available_universe.append(order["Symbol"][0])
            
			# Otherwise if there would still be a remaining amount
            else:
                
				# Updating the asset's quantity on the portfolio
                self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
                
				# Combining with the other trades
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += (order["Quantity"] * order["Price"])
            
		# Otherwise the position isn't present and shorting isn't allowed so...
        else:
            
			# Informing the user
            print("This position is not present and shorting isn't available")
            
	# Method for rebalancing the portfolio
    def Rebalance(self):
        
		# Defining a rebalancer
        rebalancer = Rebalance()
        
		# Running the TLH engine of the rebalancer
        harvest = rebalancer.Engine(self.trades, self.assets)
        
		# Objects for choosing the top stocks of our universe
        loader = DataLoader()
        screener = Screener()
        allocator = Allocator()
        
        print()
        print(self.universe)
        print(self.available_universe)

        # Only harvest if there's stuff to harvest
        if harvest != None:
            
		    # Harvesting
            for i in range(len(harvest)):
            
			    # Extracting the quantity
                quantity = self.assets[self.assets["Symbol"]==harvest.iloc[i]]["Quantity"].item()
            
			    # Harvesting the position
                self.SellAsset(harvest.iloc[i], quantity)
            
        print()
        print(self.universe)
        print(self.available_universe)
        print()
        
		# Calculating quantities for what stocks
        stocks = loader.Past(self.universe)
        screened = screener.Screen(stocks, self.size)
        allocated = allocator.Allocate(screened, self.capital, loader)
        print(allocated)

In [341]:
MyPf = Portfolio(["AAPL", "COIN", "SOFI"], 10000, 2)

[*********************100%%**********************]  3 of 3 completed


In [342]:
MyPf.assets

,Symbol,Quantity,LT Gains,ST Gains
0,filler,filler,filler,filler
1,AAPL,43.0,NaN,NaN
2,SOFI,3.0,NaN,NaN


In [343]:
MyPf.Rebalance()

[*********************100%%**********************]  3 of 3 completed

LT Gains: $500
ST Gains: $400

All Assets:
   Symbol Quantity LT Gains ST Gains
0  filler   filler   filler   filler
1    AAPL     43.0        0      400
2    SOFI      3.0      500        0

Assets At Loss:
Empty DataFrame
Columns: [Symbol, Quantity, LT Gains, ST Gains]
Index: []

Safe Assets
Empty DataFrame
Columns: []
Index: []

No losses to harvest

['AAPL', 'COIN', 'SOFI']
['COIN']

['AAPL', 'COIN', 'SOFI']
['COIN']



ValueError: Length of values (2) does not match length of index (1)

In [345]:
MyPf.universe

['AAPL', 'COIN', 'SOFI']

In [347]:
MyPf.universe[True for ]

'AAPL'

In [255]:
reb = Rebalance()

In [256]:
reb.Engine(MyPf.trades, MyPf.assets)

LT Gains: $500
ST Gains: $400

All Assets:
   Symbol Quantity LT Gains ST Gains
0  filler   filler   filler   filler
1    PLTR        5        0     -100
2    AAPL     43.0        0      400
3    SOFI      2.0      500        0

Assets At Loss:
  Symbol Quantity LT Gains ST Gains
1   PLTR        5        0     -100

Safe Assets
  Symbol Quantity LT Gains ST Gains
1   PLTR        5        0     -100

Total tax before: $130.0
Total tax after:  $110.0
Savings after harvest: $20.0

1    PLTR
Name: Symbol, dtype: object


In [27]:
MyPf.capital

0    201.7
dtype: float64

In [17]:
MyPf.assets

,Symbol,Quantity,LT Gains,ST Gains
0,filler,filler,filler,filler
1,AAPL,42.0,0,200
2,SOFI,2.0,0,0
